# Worksheet 2b: Creating future climate scenarios and analysing change

The following exercises demonstrate analyses of changes in climate simulated using two PRECIS
experiments (driven by HadCM3Q0 and ECHAM5.) As with worksheet 2a, these are just examples
of some of the analyses that you might perform using Python and Iris.

Note: The data used here has been processed in the same way as Worksheet 1. The 8 point-rim
has been removed and it has been converted from PP to netCDF format.

In [ ]:
# Ensure you have iris 1.10 or greater
import iris
import iris.coord_categorisation
import iris.quickplot as qplt
import matplotlib.pyplot as plt
import numpy as np

iris.FUTURE.netcdf_no_unlimited = True
iris.FUTURE.netcdf_promote = True

# Show plots in-line
%matplotlib inline 

## 2b.1 Calculate OND mean precipitation
First, we calculate OND mean precipitation for (a) the HadCM3Q0-driven PRECIS simulation (cahpa) and (b) the ECHAM5-driven simulation (cahpb):

In [ ]:
ROOTDIR = '/data/users/hsteptoe/pyprecis/monthly/' #this need to be set to the final agreed data location
path_in_cahpa = "practise_ppfilesmonthly/cahpa/05216/"
PATH_CLIM = ROOTDIR + 'climatology/' 

jobids = ['cahpa', 'cahpb']

for jobid in jobids:
    path_in = ROOTDIR + jobid + '/05216/'
    data = iris.load_cube(path_in + jobid + 'a.pm.2150.05216.rr8.mmday.nc')

    # in order to calculate OND mean, we divide the months into two seasons: 
    # one for OND and a second for the remaining months
    iris.coord_categorisation.add_season(data, 'time', name='seasons', seasons=('jfmamjjas','ond'))

    # Extract the data for the OND season only
    data_ond = data.extract(iris.Constraint(seasons='ond'))

    # Now calculate the mean over the OND season
    future_ond_mean = data_ond.aggregated_by(['seasons'], iris.analysis.MEAN)

    # save the OND mean as a netCDF
    iris.save(future_ond_mean, PATH_CLIM + jobid + 'a.OND.mean.future.05216.mmday.nc')
    print('{} saved to {}').format(jobid, PATH_CLIM)

Within the loop, we have created two cubes. This is the seasonal OND constrained cube:

In [ ]:
print data_ond

 And this is the seasonal OND mean cube for the future period 2021-2050:

In [ ]:
print future_ond_mean

Remember that the loop has created and saved two cubes, one for each jobid

## 2b.2 Find OND anomalies
Next, we subtract the baseline mean from the future for OND to get the change in precipitation (or anomly) from both simulations.  The changes are also converted to percentages:

In [ ]:
for jobid in jobids:
    # Load the baseline cube
    OND_baseline = iris.load_cube(PATH_CLIM + jobid + 'a.OND.mean.baseline.05216.mmday.nc')
    # Set the correct units
    OND_baseline.units = "mm day-1"
    # Load the future cube
    OND_future = iris.load_cube(PATH_CLIM + jobid + 'a.OND.mean.future.05216.mmday.nc')
    # Subtract the baseline cube from the future cube
    diff = iris.analysis.maths.subtract(OND_future, OND_baseline)
    # Save the resulting cube
    iris.save(diff, PATH_CLIM + jobid + 'a.OND.mean.diff.05216.nc')
    # Find the percentage change
    pcent_change = iris.analysis.maths.multiply(iris.analysis.maths.divide(diff, OND_baseline), 100)
    # And save this too
    iris.save(pcent_change, PATH_CLIM + jobid + 'a.OND.mean.diff.05216.perc.nc')
    print('{} saved to: {}'.format(jobid, PATH_CLIM))

Repeat the calculations above for **temperature**. First, we calculate the OND mean temperatures:

In [ ]:
time_periods = {'baseline':'6190', 'future':'2150'}

for jobid in jobids:
    for period in time_periods.keys():
        path_in = ROOTDIR + jobid + '/03236/'
        # Load the data
        data = iris.load_cube(path_in + jobid + 'a.pm.' + time_periods[period] + '.03236.rr8.nc')
        
        # in order to calculate OND mean, we divide the months into two seasons, 
        # one for OND and a second for the remaining months
        iris.coord_categorisation.add_season(data, 'time', name='seasons', seasons=('jfmamjjas','ond'))

        # Extract the data for the OND season only
        data_ond = data.extract(iris.Constraint(seasons='ond'))

        # Now calculate the mean over the OND season
        ond_mean = data_ond.aggregated_by(['seasons'], iris.analysis.MEAN)
        ond_mean.convert_units('celsius')

        # save the OND mean as a netCDF
        iris.save(ond_mean, PATH_CLIM + jobid + 'a.OND.mean.' + time_periods[period] + '.03236.nc')
        print('{}:{} saved to {} directory'.format(jobid, period, PATH_CLIM))

Next, we calculate the differences between the baseline and future periods.

In [ ]:
for jobid in jobids:
    OND_baseline = iris.load_cube(PATH_CLIM + jobid + 'a.OND.mean.baseline.03236.nc')
    OND_future = iris.load_cube(PATH_CLIM + jobid + 'a.OND.mean.future.03236.nc')
    diff = iris.analysis.maths.subtract(OND_future, OND_baseline)
    iris.save(diff, PATH_CLIM + jobid + 'a.OND.mean.diff.03236.nc')
    print('{} saved to {}'.format(jobid, PATH_CLIM))

## 2b.3 Plot precipitation and temperature

Plot changes to precipitation (in %) and temperature (in deg.C)

In [ ]:
# Create a figure of the size 12x12 inches
plt.figure(figsize=(12, 12))

# Read in the percentage changes in precipitation
for n, jobid in enumerate(jobids):
    pcent_change = iris.load_cube(PATH_CLIM + jobid + 'a.OND.mean.diff.05216.perc.nc')
    degc_change = iris.load_cube(PATH_CLIM + jobid + 'a.OND.mean.diff.03236.nc')

    # Remove extra time dimension
    pcent_change = iris.util.squeeze(pcent_change)
    degc_change = iris.util.squeeze(degc_change)
    
    plot_num = n*2 + 1
    plt.subplot(2, 2, plot_num) # Create a new subplot with 2 rows, 2 columns, 1st plot
    qplt.pcolormesh(pcent_change, vmax=30, vmin=-30, cmap='BrBG')
    plt.title(jobid + ' precipitation change (%)')
    ax = plt.gca()              # gca function that returns the current axes
    ax.coastlines()             # adds coastlines defined by the axes of the plot
    ax.gridlines()              # adds x and y grid lines to the plot

    plt.subplot(2, 2, plot_num+1)
    qplt.pcolormesh(degc_change, vmax=2.5, vmin=0, cmap='Reds')
    plt.title(jobid + ' temperature change ($\degree$C)')
    ax = plt.gca()
    ax.coastlines()
    ax.gridlines()

plt.tight_layout()          # automatically adjusts subplot(s) to fit in to the figure area
plt.show()

## 2b.4 Future time series

Calculate and then plot a 2021-2050 monthly time series of precipitation anomalies for land
points only relative to the 1961-1990 baseline monthly mean. Do this for both cahpa and cahpb.


In [ ]:
# Read in the land-sea mask. 
# The cube data array has a mask associated with it which we'll use to mask out ocean points.
land_sea_mask = iris.load_cube('../data/landmask.nc')

for jobid in jobids:
    # Read in original data for baseline and future
    path_in = ROOTDIR + jobid + '/05216/'
    baseline = iris.load_cube(path_in + jobid + 'a.pm.6190.05216.rr8.mmday.nc')
    future = iris.load_cube(path_in + jobid + 'a.pm.2150.05216.rr8.mmday.nc')
    

    # Apply land mask. Probably not most efficient method. baseline / future data arrays are changed to masked type
    baseline.data = ma.array(baseline.data, mask=baseline.data*land_sea_mask.data.mask[np.newaxis, :,:])
    future.data = ma.array(future.data, mask=future.data*land_sea_mask.data.mask[np.newaxis, :,:])

    # Calculate mean values over land points
    baseline_land = baseline.collapsed(['grid_longitude', 'grid_latitude'], iris.analysis.MEAN)
    future_land = future.collapsed(['grid_longitude', 'grid_latitude'], iris.analysis.MEAN)

    # Save the area averaged monthly differences
    iris.save(baseline_land, PATH_CLIM + jobid + 'a.baseline.tseries.05216.nc')
    iris.save(future_land, PATH_CLIM + jobid + 'a.future.tseries.05216.nc')

    # Subtract baseline from future and save
    diff = future_land.copy()
    diff.data = future_land.data - baseline_land.data

    # Save the area averaged monthly differences
    iris.save(diff, PATH_CLIM + jobid + 'a.future.tseries.diff.05216.nc')
    print 'The cube has been saved to ' + PATH_CLIM + jobid + 'a.future.tseries.diff.05216.nc'

Plot the precipitation anomalies of cahpa and cahpb

In [ ]:
path_clim = "/data/users/ssadri/pyprecis/monthly/climatology/"

# Read in the monthly series
cahpa = iris.load_cube(path_clim + 'cahpaa.future.tseries.diff.05216.nc')
cahpb = iris.load_cube(path_clim + 'cahpba.future.tseries.diff.05216.nc')

# Plot the two series on the same figure
cahpa_plot = qplt.plot(cahpa,color = 'blue' ,  label = 'cahpa')
print "time serries plot for: cahpa"
qplt.show()
cahpb_plot = qplt.plot(cahpb, color = 'blue',  label = 'cahpb')
print "time serries plot for: cahpb"
qplt.show()